In [1]:
import pandas as pd

In [2]:
expert_agent = pd.read_csv("data/data_generation_mpc_110_190_6_all.csv")

In [3]:
import joblib

# Load X_scaler
X_scaler = joblib.load('data/scalers/X_scaler.pkl')

# Load y_scaler
y_scaler = joblib.load('data/scalers/y_scaler.pkl')


In [4]:
import torch
import torch.nn as nn

In [5]:
class LitNeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 50)
        self.fc2 = nn.Linear(50, 10)
        self.fc3 = nn.Linear(10, 2)
        self.train_loss = []
        self.val_loss = []

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
# Load the model state dictionary
model_state_dict = torch.load('data/model/trained_model.pth')

# Create an instance of the model
model = LitNeuralNet()

# Load the state dictionary into the model
model.load_state_dict(model_state_dict)


<All keys matched successfully>

In [7]:
def test_model_with_custom_inputs(inflow, height, X_scaler, y_scaler, model):
    custom_inputs =  pd.DataFrame({"inflow": [inflow], 
                  "height": [height]})
    custom_inputs_scaled = X_scaler.transform(custom_inputs)
    custom_inputs_scaled_tensor = torch.tensor(custom_inputs_scaled, dtype=torch.float32)
    with torch.no_grad():
        output = model(custom_inputs_scaled_tensor)
        output_unscaled = y_scaler.inverse_transform(output)
        return custom_inputs_scaled_tensor , output_unscaled

In [13]:
pumps_speeds = test_model_with_custom_inputs(100, 1.6, X_scaler, y_scaler, model)[1]
print(pumps_speeds)

[[953.68628747 771.97295517]]


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [10]:
from auto_LiRPA import BoundedModule, BoundedTensor
from auto_LiRPA.perturbations import *

No CUDA runtime is found, using CUDA_HOME='/usr'


In [11]:
custom_inputs_scaled = X_scaler.transform(expert_agent[["inflow", "height"]])
X = torch.tensor(custom_inputs_scaled, dtype=torch.float32)

In [33]:
# Step 4: Apply Auto_LiRPA
X0, yhat = test_model_with_custom_inputs(100, 1.6, X_scaler, y_scaler, model)
epsilon = 0.1
# Create perturbation
perturbation = PerturbationLpNorm(norm=np.inf, eps=epsilon)
# Convert the model to a Bounded Module
bounded_model = BoundedModule(model, torch.empty_like(X), bound_opts={"forward_mode": "interval"})
#Create a BoundedTensor from the input and perturbation
bounded_input = BoundedTensor(X0, perturbation)
#Compute bounds using the compute_bounds() method.
print('Bounding method: backward (CROWN, DeepPoly)')
with torch.no_grad():  # If gradients of the bounds are not needed, we can use no_grad to save memory.
  lb, ub = bounded_model.compute_bounds(x=(bounded_input,), method='CROWN')

def print_bounds(lb, ub, y_hat, y_scaler):
    lb = lb.detach().cpu().numpy()
    ub = ub.detach().cpu().numpy()
    lb = y_scaler.inverse_transform(lb)
    ub = y_scaler.inverse_transform(ub)
    dataset = []
    pump_labels = ["pump1", "pump2"]

    for j in range(2):
        lower_bound = lb[0][j]
        upper_bound = ub[0][j]
        row = {
            "pump label": pump_labels[j],
            "pump_rpm estimated": y_hat[0][j],
            "lower_bound": lower_bound,
            "upper_bound": upper_bound
        }
        dataset.append(row)

    return pd.DataFrame(dataset)
        
# Auxillary function to print bounds.
print_bounds(lb, ub, yhat, y_scaler)

Bounding method: backward (CROWN, DeepPoly)


,pump label,pump_rpm estimated,lower_bound,upper_bound
0,pump1,953.686287,906.687744,998.488708
1,pump2,771.972955,717.251709,824.163513
